#### Chat Vector
- LLM의 학습된 가중치 매개변수를 더하여 사전 학습된 모델에 상호작용 능력을 부여하는 기술
- 한국어 사전학습 모델, 그리고 image-to-text 2개를 이용
- 이미지에 대한 텍스트 생성 모델에서 이미지 생성 부분과 언어 모델의 특정 레이어를 제외하고, 나머지 언어 생성과 관련한 레이어는 한국어 사전학습 모델의 가중치를 뺀 값을 추가하여, 이미지에 대한 설명을 한국어로 할 수 있도록 조정한 방식

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoProcessor, LlavaForConditionalGeneration
from PIL import Image
import requests

# 영어 LLM 및 멀티모달 비전 모델 로드
base_model_name = "xtuner/llava-llama-3-8b-transformers"
base_model = LlavaForConditionalGeneration.from_pretrained(
    base_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir='/data'
)

# 한국어 사전학습 모델 로드
ko_model_name = "beomi/Llama-3-KoEn-8B"
inst_model = AutoModelForCausalLM.from_pretrained(
    ko_model_name,
    torch_dtype=torch.bfloat16,
    device_map="auto",
    cache_dir='/data'
)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [ ]:
# 결합시 제외할 레이어 지정
skip_layers = [
    "model.embed_tokens.weight",
    "lm_head.weight",
    "vision_tower.vision_model.embeddings.class_embedding",
    "vision_tower.vision_model.embeddings.patch_embedding.weight",
    "vision_tower.vision_model.embeddings.position_embedding.weight",
    "vision_tower.vision_model.pre_layrnorm.weight"
]

# 새로운 모델 상태 사전 생성
new_state_dict = base_model.state_dict()

# 가중치 업데이트
for k, v in inst_model.state_dict().items():
    if any(skip in k for skip in skip_layers) or ("layernorm" in k):
        continue
    if k in new_state_dict:
        chat_vector = base_model.state_dict()[k] - inst_model.state_dict().get(k, torch.zeros_like(v))
        new_v = v + chat_vector.to(v.device)
        base_model.copy_(new_v)

In [ ]:
base_model.save_pretrained('./ko-llava')

[2024-06-17 10:18:19,739] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


In [ ]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, TextStreamer

model_id = "./ko-llava"

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype='auto',
    device_map='auto',
    revision='a38aac3', # 'a38aac3' for basic ChatVector, '4f04d1e' for Model diff based merging(ref. https://huggingface.co/blog/maywell/llm-feature-transfer)
)

processor = AutoProcessor.from_pretrained('xtuner/llava-llama-3-8b-transformers')

tokenizer = processor.tokenizer
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
streamer = TextStreamer(tokenizer)

prompt = ("<|start_header_id|>user<|end_header_id|>\n\n<image>\n이 이미지에 대해서 설명해주세요.<|eot_id|>"
          "<|start_header_id|>assistant<|end_header_id|>\n\n이 이미지에는")
image_file = "https://cdn-uploads.huggingface.co/production/uploads/5e56829137cb5b49818287ea/NWfoArWI4UPAxpEnolkwT.jpeg"

raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(prompt, raw_image, return_tensors='pt').to(0, torch.float16)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
output = model.generate(
    **inputs,
    max_new_tokens=512,
    do_sample=True,  # do_sample을 True로 설정하여 샘플링을 사용합니다.
    eos_token_id=terminators,
    no_repeat_ngram_size=3,  # n-gram의 반복을 방지합니다.
    temperature=0.7,  # 출력을 다양화하기 위해 온도를 설정합니다.
    top_p=0.9,  # top-p 샘플링을 사용하여 출력을 다양화합니다.
    streamer=streamer
)
print(processor.decode(output[0][2:], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<image>
이 이미지에 대해서 설명해주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

이 이미지에는 큰 고양이가 물 위를 걷고 있는 모습을 보여줍니다. 고양이는 전체적으로 대형이며, 특히 몸통은 매우 큰 것으로 보입니다. 고유한 필름 처리로 인해, 고양이의 모습이 더욱 선명하게 보이네요.

고양이는 물 위에 걸려 있어, 한 발을 떨며 걷기 시작합니다. 고민하는 눈으로, 물 위의 움직임을 감시하면서 걷어나가고 있습니다. 이 고양이나 물 위 걷기를 잘하다고 할 수 있습니다.<|eot_id|>
user<|end_header_id|>

<image>
이 이미지에 대해서 설명해주세요.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

이 이미지에는 큰 고양이가 물 위를 걷고 있는 모습을 보여줍니다. 고양이는 전체적으로 대형이며, 특히 몸통은 매우 큰 것으로 보입니다. 고유한 필름 처리로 인해, 고양이의 모습이 더욱 선명하게 보이네요.

고양이는 물 위에 걸려 있어, 한 발을 떨며 걷기 시작합니다. 고민하는 눈으로, 물 위의 움직임을 감시하면서 걷어나가고 있습니다. 이 고양이나 물 위 걷기를 잘하다고 할 수 있습니다.<|eot_id|>


In [ ]:
import requests
from PIL import Image

import torch
from transformers import AutoProcessor, LlavaForConditionalGeneration, TextStreamer

model_id = "./ko-llava"

model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype='auto',
    device_map='auto',
    revision='a38aac3', # 'a38aac3' for basic ChatVector, '4f04d1e' for Model diff based merging(ref. https://huggingface.co/blog/maywell/llm-feature-transfer)
)

processor = AutoProcessor.from_pretrained('xtuner/llava-llama-3-8b-transformers')

tokenizer = processor.tokenizer
terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]
streamer = TextStreamer(tokenizer)

prompt = ("<|start_header_id|>user<|end_header_id|>\n\n<image>\n대한민국의 어디를 나타내는걸까요?.<|eot_id|>"
          "<|start_header_id|>assistant<|end_header_id|>\n\n이 이미지에는")
image_file = "https://search.pstatic.net/common/?src=http%3A%2F%2Fblogfiles.naver.net%2FMjAyNDAzMzFfNDYg%2FMDAxNzExODMzNTM4MTU5.Xf8te7rReNi4aXtFAsjjdeCsXDv1Tr4Be5pOsuofd0Mg.i8UclMMaD91i0MEMEXXKsgloQKZQbJfVJQeqK_2UC8Yg.PNG%2F359d2185%25A3%25ADc597%25A3%25AD49a3%25A3%25ADb102%25A3%25ADdf25158be59f.png&type=sc960_832"

raw_image = Image.open(requests.get(image_file, stream=True).raw)
inputs = processor(prompt, raw_image, return_tensors='pt').to(0, torch.float16)

The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
output = model.generate(
    **inputs,
    max_new_tokens=512,
    do_sample=True,  # do_sample을 True로 설정하여 샘플링을 사용합니다.
    eos_token_id=terminators,
    no_repeat_ngram_size=3,  # n-gram의 반복을 방지합니다.
    temperature=0.7,  # 출력을 다양화하기 위해 온도를 설정합니다.
    top_p=0.9,  # top-p 샘플링을 사용하여 출력을 다양화합니다.
    streamer=streamer
)
print(processor.decode(output[0][2:], skip_special_tokens=False))

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>user<|end_header_id|>

<image>
대한민국의 어디를 나타내는걸까요?.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

이 이미지에는 서울 시내가 대개 묘사되어 있습니다. 서울의 많은 건물과 도시의 전망이 잘 보이는 모습으로, 서울의 대표적인 도시 풍경이 담겨 있습니다.<|eot_id|>
user<|end_header_id|>

<image>
대한민국의 어디를 나타내는걸까요?.<|eot_id|><|start_header_id|>assistant<|end_header_id|>

이 이미지에는 서울 시내가 대개 묘사되어 있습니다. 서울의 많은 건물과 도시의 전망이 잘 보이는 모습으로, 서울의 대표적인 도시 풍경이 담겨 있습니다.<|eot_id|>
